In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
class BNEPFPISolver(): 
    
    def __init__(self, I, J, F, Jf, Bi, a, V, c, f_tol=1.0e-6, r_tol=1.0e-6, max_iter=1000): 
        
        """
        
        I:  number of individuals (integer)
        J:  number of products (integer)
        F:  number of firms (integer)
        Jf: number of products per firm (F vector)
        Bi: "budgets" for each individual (I vector)
        a:  price sensitivity for each individual (I vector)
        V:  nonprice utility for each individual/product (I x J matrix)
        c:  costs for each product (J vector)
        
        
        f_tol: the convergence tolerance
        max_iter: the maximum number of iterations
        
        """
        
        # problem data
        self.I  = I  # number of individuals
        self.J  = J  # number of products
        self.F  = F  # number of firms
        self.Jf = Jf # list of the number of products per firm
        self.b  = Bi # individual incomes
        self.a  = a  # price sensitivity
        self.V  = V  # "fized" portion of utility (non-price part)
        self.c  = c  # (unit) costs for each product
        
        # iteration options
        self.f_tol    = f_tol
        self.r_tol    = r_tol
        self.max_iter = max_iter
        
        # internal data
        
        # max income and max income index
        self.maxbi = np.argmax(Bi)
        self.maxb  = Bi[self.maxbi]
        
        # indices segmenting firm blocks
        self.Fis = [ None for f in range(F) ]
        self.Fis[0] = range(0,self.Jf[0])
        for f in range(1,F):
            self.Fis[f] = range(self.Fis[f-1][-1]+1,self.Fis[f-1][-1]+1+self.Jf[f])
        
        # other storage
        self.m    = np.zeros(J)      # markups (prices minus costs)
        self.pr   = np.zeros(self.F) # profits (convenience only)
        self.U    = np.zeros((I,J))  # utilities (for each individual/product)
        self.DpU  = np.zeros((I,J))  # price derivatives of utilities (for each)
        self.DppU = np.zeros((I,J))  # second price derivatives of utilities (fe)
        self.PL   = np.zeros((I,J))  # logit probabilities (idiosyncratic)
        self.P    = np.zeros(J)      # mixed probabilities
        self.L    = np.zeros(J)      # "Lambda" operator, diagonal matrix
        self.G    = np.zeros((J,J))  # "Gamma" operator, dense matrix
        self.z    = np.zeros(J)      # "zeta" map
        self.phi  = np.zeros(J)      # "phi" map, what we want to zero

In [ ]:
def utilities(self, p):
        """
        
        Define self.U[i,j], self.DpU[i,j], and self.DppU[i,j]
        for all individuals i and products j as a function of 
        prices p[j], accounting for finite purchasing power 
        in the sense that if p[j] >= b[i], for "budget" b[i], 
        
               U[i,j] ~ -Inf (-1e20 here)
             DpU[i,j] = -Inf (set to zero here)
            DppU[i,j] = lim_{ p -> b[i] } [ DppU[i,j](p) / DpU[i,j](p)^2 ]
            
        For example, if u = a[i] log(b[i]-p) + w[i,j], 
        
               U[i,j] =   a[i] log(b[i]-p) + w[i,j]
             DpU[i,j] = - a[i] / (b[i]-p)
            DppU[i,j] = - a[i] / (b[i]-p)^2
            
        when p < b[i] but 
        
               U[i,j] = - Inf    ** but store -1.0e20 **
             DpU[i,j] = - Inf    ** but store 0 **
            DppU[i,j] = - 1/a[i]
            
        when p >= b[i]. 
        
        This weird storage format is just because we need that 
        limiting ratio of first/second derivatives for the right 
        extension of the fixed point map. See the paper. 
        
        The setting of DpU[i,j] to zero is basically an expression
        of the condition DpU[i,j] PL[i,j] -> 0 as p[j] -> b[i]. 
        This is discussed in Assumption 1 of the paper, but 
        basically ensures sufficient continuity for use of deriv-
        atives for analyzing equilibrium. By setting DpU[i,j] to 
        zero, we avoid numerical issues from other values. 
        
        Technically, we should probably build some confidence that
        values like DpU[i,j] PL[i,j] are _numerically_ continuous 
        as well. 
        
        """
        
        T = self.b.reshape(I,1) - p.reshape(1,J)
        for i in range(self.I): 
            for j in range(self.J): 
                if T[i,j] > 0.0: # tolerance, not just > 0.0? 
                    self.U[i,j]    =   self.a[i] * np.log( T[i,j] ) + self.V[i,j]
                    self.DpU[i,j]  = - self.a[i] / T[i,j]
                    self.DppU[i,j] =   self.DpU[i,j] / T[i,j]
                else:
                    self.U[i,j]    = - 1.0e20
                    self.DpU[i,j]  =   0.0
                    self.DppU[i,j] = - 1.0/self.a[i]

In [3]:
 def probabilities(self):
        self.PL , self.P = np.zeros((I,J)) , np.zeros(J)
        uimax = np.maximum( 0 , np.max( self.U , axis=1 ) )
        for i in range(self.I): 
            self.PL[i,:]  = np.exp( self.U[i,:] - uimax[i] )
            self.PL[i,:] /= np.exp( - uimax[i] ) + self.PL[i,:].sum()
        self.P = np.sum( self.PL , axis=0 ) / self.I
        


In [4]:
  def lamgam(self):
        DpUPL = self.DpU * self.PL
        self.L = np.sum( DpUPL , axis=0 ) / self.I
        self.G = self.PL.T.dot( DpUPL ) / self.I

In [ ]:
def zeta(self, p, verbose=False):
        
        # z <- inv(diag(LAMp)) * ( \tilde{GAMp}' * m - P )
        # for all prices < maxinc, corrected otherwise
        
        self.utilities(p)
        self.probabilities()
        self.lamgam()
        
        # compute markups (not used elsewhere)
        self.m = p - self.c
        
        # z <- \tilde{GAMp}' * m - P
        for f in range(self.F): 
            fi = self.Fis[f]
            # self.z[fi] = self.G[fi,fi].T.dot( self.m[fi] ) - self.P[fi]
            self.z[fi] = - self.P[fi]
            for j in fi: 
                for k in fi: 
                    self.z[j] += self.G[k,j] * self.m[k]
            
        # nominally z <- inv(diag(LAMp)) * z, but with corrections 
        # for products whose prices are above the population limit
        # on incomes. The correction is
        # 
        #     z[j] = omega[maxinci,j] * ( p[j] - maxinc ) + PL[maxinci,{f}]' * m[{f}]
        # 
        # for j : p[j] > maxb
        for f in range(self.F):
            fi = self.Fis[f]
            prFmi = self.PL[self.maxbi,fi].T.dot( self.m[fi] )
            for j in fi:
                if p[j] > self.maxb: # correction term - price j is too high
                    print( f"WARNING ({self.iter}) -- {p[j]} > {self.maxb} correcting zeta for product {j}" )
                    self.z[j] = self.DppU[self.maxbi,j] * ( p[j] - self.maxb ) + prFmi
                else:
                    if self.L[j] < 0.0: 
                        # some tolerance would be better than "0", like 
                        # LAM[j] < -1.0e-10 (just as an example)
                        self.z[j] /= self.L[j]
                    else: 
                        print( f"WARNING ({self.iter}) -- p[{j}] = {p[j]} < {self.maxb} = maxinc but LAMp[{j}] = {self.L[j]}.")
                        # use a modification of extended map instead of what is calculated above
                        # z[j] = PL[maxinci,{f}]' * m[{f}]
                        self.z[j] = prFmi
                        # we exclude the "DppU[I*j+maxinci] * ( p[j] - maxinc )" term expecting
                        # p[j] to be at least close to maxinc
        
        # compute phi = p - c - z also:
        self.phi = p - self.c - self.z